<a href="https://colab.research.google.com/github/pgordin/GraphsSN2024_1/blob/main/Graphs8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Graphs & Social Networks &mdash; lab material
This notebook contains a definition of a python class *Graph*, containing functions described during labs.

Package import.

In [1]:
import numpy as np
from random import random, seed
from copy import deepcopy
from queue import PriorityQueue, deque

## Simple functions from the first classes
This is left just as an example (to compare the behaviour).

In [2]:
def print_matrix(vertices, matrix):
  """
  Printing a graph given by adjacency matrix
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i, j]:
        print(" ", vv[j], end="")
    print("")

def print_dict(graph):
  """
  Printing of a graph (given as a dictionary/neighbouring list)
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")

## Class *Graph*

In [3]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_matrix(cls, matrix, vertices = None):
        if (vertices is None) or (len(vertices) != len(matrix)):
            vertices = [*range(1, len(matrix) + 1)]
        return cls.from_dict(cls._matrix_to_dict(matrix, vertices))

    # two private methods matrix <-> dictionaries
    def _matrix_to_dict(matrix, vertices: list) -> dict:
        """
        Converts a graph given as an adjacency matrix to a graph in dict form.
        """
        res_dict = {}
        for i, v in enumerate(vertices):
            neighbours = [vertices[j] for j, edge in enumerate(matrix[i]) if edge]
            res_dict[v] = neighbours
        return res_dict

    def _dict_to_matrix(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to its adjacency matrix.
        """
        n = len(_dict)
        vertices = [*_dict.keys()]
        matrix = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (vertices.index(u), vertices.index(v))
            for u, row in _dict.items() for v in row
        ]:
            matrix[u][v] += 1
        return matrix

    def vertices(self) -> list:
        """
        Returns list of vertices of the graph.
        """
        return [*self.graph.keys()]

    def matrix(self) -> np.array:
        """
        Returns the adjacency matrix of the graph.
        """
        return self._dict_to_matrix(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    # The following is for free thanks to the above
    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")  # open textfile for writing
        file.write(str(self))
        file.close()

    # Modyfying graphs
    def add_vertex(self, vertex):
        """
        Adds a new vertex to the graph.
        """
        if vertex not in self.graph:
            self.graph[vertex] = []

    def del_vertex(self, vertex):
        """
        Removes a vertex from the graph.
        """
        if vertex in self.graph:
            self.graph.pop(vertex)
            for u in self.graph:
                if vertex in self.graph[u]:
                    self.graph[u].remove(vertex)

    def add_arc(self, arc):
        """
        Given pair of vertices (arc variable) add an arc to the graph
        We consider simple, directed graphs.
        """
        u, v = arc
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Given pair of vertices (edge variable) add an edge to existing graph.
        We consider simple, undirected graphs, as symmetric digraphs without loops.
        """
        u, v = edge
        if u == v:
            raise ValueError("Loops are not allowed!")
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    # reading from a file
    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Read the graph from file, that in each line contains either
        the description of a vertex (one word) or
        the description of an edge/arc (at least 2 words).
        The resulting graph is returned as a neighbourhood list.
        Variable "filename" contains the whole path to the file.
        """
        graph = Graph()
        file = open(filename, "r")          # open the file to read
        for line in file:                   # for each line of the file
          words = line.strip().split()      # splits the line into words
          if len(words) == 1:               # one word - vertex description
            graph.add_vertex(words[0])
          elif len(words) >= 2:             # at least two words, first two are the edge description
            if directed:
              graph.add_arc([words[0], words[1]])
            else:
              graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(n: int, p: float):
        """
        Creates a random graph in G(n, p) model.
        """
        rand_graph = Graph()
        for i in range(1, n + 1):
            rand_graph.add_vertex(i)
            for j in range(1, i):
                if random() < p:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(n: int):
        """
        Creates a cycle C_n on n vertices
        """
        cycle = Graph()
        for i in range(n-1):
          cycle.add_edge([i+1, i+2])
        cycle.add_edge([1, n])
        return cycle


    def Prufer(self):
      """
      Returns the Prufer code of a tree.
      It is necessary that the graph is a tree (it is not checked).
      Result is given as a string (empty for trees on 1 or 2 vertices).
      """
      tr = deepcopy(self.graph)   # copy of a tree, as we destroy it
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):
          if len(tr[x]) == 1:   # least leaf
            break
        v = tr[x][0]            # the unique neighbour of x
        code = code + f"{v} "
        tr[v].remove(x)         # remove x from neighbours of v
        tr.pop(x)               # remove x from the tree
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Creating a tree from a Prufer code.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # code as a list of numbers
        n = len(clist) + 2                  # number of vertices
        vert = [*range(1, n+1)]             # list of numbers 1..n
        for v in vert:
          tree.add_vertex(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # x - least leaf
              break
          v = clist.pop(0)    # remove the first element from the code - the neighbour of x
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree


    def connected_components(self):
      """
      Looks for connected components of undirected graph.
      Returns a list of its vertex-sets.
      Remark: the first element contains the set of all graph vertices
      """
      def DFS(u):
        """
        Deep first search (as internal method).
        """
        for w in self.graph[u]:
          if w not in VT[0]:      # w - not visited yet
            VT[0].add(w)          # already visited
            VT[-1].add(w)         # w - in the last connected component
            DFS(w)

      """
      VT - list of vertex sets VT[i] for i > 0 - is a vertex set of i-th connected component
      VT[0] - is a vertex set of the spanning forest (or during the algorithm list of visited vertices).
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:      # v is not visited
          VT[0].add(v)
          VT.append(set([v]))   # statring point of new conected component
          DFS(v)
      return VT

    def preorder(self, v, visited=None):
        # first we print a given vertex, then we traverse the subtree rooted in it
        """
        Prints the vertices of the graph in preorder traversal starting from vertex v.
        """
        if visited is None:
            visited = set()

        # Visit the current node
        visited.add(v)
        print(v, end=' ')

        # Recursively visit all unvisited neighbors
        for neighbor in self.graph[v]:
            if neighbor not in visited:
                self.preorder(neighbor, visited)


    def postorder(self, v, visited=None):
        # postorder — first we traverse the subtree rooted a given vertex, then we list the vertex
        """
        Prints the vertices of the graph in postorder traversal starting from vertex v.
        """
        if visited is None:
            visited = set()

        # Mark the current node as visited
        visited.add(v)

        # Recursively visit all unvisited neighbors first
        for neighbor in self.graph[v]:
            if neighbor not in visited:
                self.postorder(neighbor, visited)

        # Print the current node after visiting its neighbors
        print(v, end=' ')


    def connected_components_graphs(self):
        """
        Returns a list of Graph objects, each representing a connected component of the original graph.
        """
        components = self.connected_components()  # Get the list of connected components as vertex sets
        component_graphs = []

        for component in components[1:]:  # Skip the first element as it contains all vertices
            subgraph = Graph()            # Create a new Graph instance
            for vertex in component:
                subgraph.graph[vertex] = self.graph[vertex]
#           equivalent
#                subgraph.add_vertex(vertex)
#                for neighbor in self.graph[vertex]:
#                    if neighbor in component:
#                        subgraph.add_edge([vertex, neighbor])
            component_graphs.append(subgraph)

        return component_graphs


    def ConnectedComponentsBFS(self):
        """
        Connected components via BFS (homework, improved) - returns list of graphs
        """
        visited = set()  # keep track of visited vertices
        components = []  # store connected components

        for start_vertex in self.graph:
            if start_vertex not in visited:
                # if vertex was not visited, create a new component
                component = set()
                queue = deque([start_vertex])  # BFS queue

                while queue:
                    vertex = queue.popleft()    # take the vertex from the bottom
                    if vertex not in visited:
                        visited.add(vertex)
                        component.add(vertex)
                        # add all unvisited neighbors to the queue
                        for neighbor in self.graph[vertex]:
                            if neighbor not in visited:
                                queue.append(neighbor)
                # store the current connected component (as graph)
                subgraph = Graph()            # Create a new Graph instance
                for vertex in component:
                  subgraph.graph[vertex] = self.graph[vertex]
                components.append(subgraph)
        return components

    def ConnectedComponentsGraphs(self):
        """
        Connected components via DFS nonrecursive (homework, corrected) -
        returns list connected subgraphs
        """
        con_com = []
        visited = set()

        def dfs(v, component):
            stack = [v]
            while stack:
                u = stack.pop()
                if u not in visited:
                    visited.add(u)
                    component[u] = [neighbor for neighbor in self.graph[u] ] # correction: if neighbor not in visited]
                    stack.extend(component[u])

        # back to the main attraction
        for vertex in self.graph:
            if vertex not in visited:
                component = {}                  # creating a new component as an adj list
                dfs(vertex, component)          # find all vertices in this component
                con_com.append(Graph.from_dict(component))  #converting to Graph object and append

        return con_com


    @staticmethod
    def random_bipartite_graph(m, n, p):
        """
        Generates a random bipartite graph with m + n vertices.
        The two sets U and V have m and n vertices respectively.
        Each edge between a vertex in U and a vertex in V is included with probability p.
        """
        bipartite_graph = Graph()
        U = [f"U{i}" for i in range(1, m + 1)]  # Label vertices in set U as U1, U2, ..., Um
        V = [f"V{i}" for i in range(1, n + 1)]  # Label vertices in set V as V1, V2, ..., Vn

        # Add vertices to the graph
        for vertex in U + V:
            bipartite_graph.add_vertex(vertex)

        # Add edges between vertices in U and V with probability p
        for u in U:
            for v in V:
                if random.random() < p:
                    bipartite_graph.add_edge([u, v])

        return bipartite_graph


    def distance(self, v):
      """
      Computes distances from vertex v to each vertex reachable from v.
      It uses a BFS approach.
      Result is given as a dictionary of distances
      """
      dist = {v:0}    # starting point of a dictionary
      queue = [v]
      while len(queue) > 0:
        u = queue.pop(0)
        for w in self.graph[u]:
          if not w in dist:
            dist[w] = dist[u] + 1
            queue.append(w)
      return dist

    def strong_connected_components(self):
        """
        Finds strongly connected components (SCCs) using Kosaraju's algorithm.
        Returns a list of strongly connected components, each component is a set of vertices.
        """
        def dfs(v, visited, stack):
            visited.add(v)
            for neighbor in self.graph[v]:
                if neighbor not in visited:
                    dfs(neighbor, visited, stack)
            stack.append(v)

        def reverse_graph():
            reversed_graph = Graph()
            for v in self.graph:
                for neighbor in self.graph[v]:
                    reversed_graph.add_arc([neighbor, v])  # Reverse the direction of the edges
            return reversed_graph

        def dfs_reverse(v, visited, component):
            visited.add(v)
            component.add(v)
            for neighbor in reversed_graph.graph[v]:
                if neighbor not in visited:
                    dfs_reverse(neighbor, visited, component)

        stack = []
        visited = set()

        for vertex in self.graph:
            if vertex not in visited:
                dfs(vertex, visited, stack)

        reversed_graph = reverse_graph()

        visited.clear()
        scc_list = []

        while stack:
            v = stack.pop()
            if v not in visited:
                component = set()
                dfs_reverse(v, visited, component)
                scc_list.append(component)

        return scc_list

## Class *WeightedGraph*

This class uses two dictionaries: *graph* and *weights* that
should be consisted. The former contains edge weights (indexed by pairs $(u, v)$ of neighbouring vertices).  

TODO:
*   creating weighted graph from (ordinary) graph
*   creating ordinary graph from weighted one
*   checking consistency
*   making both dictionaries consistent


In [4]:
class WeightedGraph(Graph):
    def __init__(self, graph=None, weights=None):
        if weights is None:
            weights = {}
        self.weights = weights
        super().__init__(graph)

    # redefinition of print for objects of class WeightedGraph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" ({u}, {self.weights[(v, u)]})"
            res += "\n"
        return res

    # Modyfying graphs
    # add_vertex - the inherited one is enough now

    def del_vertex(self, vertex):
        """
        Removes a vertex from the graph. Assumes consistency of both dicts
        """
        if vertex in self.graph:
            for u in self.graph[vertex]:
              self.weights.pop((vertex, u))
            self.graph.pop(vertex)
            for u in self.graph:
                if vertex in self.graph[u]:
                    self.graph[u].remove(vertex)
                    self.weights.pop((u, vertex))

    def add_arc(self, arc, weight = 1):
        """
        Given pair of vertices (arc variable) add an arc to the graph
        We consider simple, directed graphs.
        If the arc already exists this operation changes its weight to a given one.
        """
        u, v = arc
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        self.weights[(u,v)] = weight

    def add_edge(self, edge: list, weight = 1):
        """
        Given pair of vertices (edge variable) add an edge to existing graph.
        We consider simple, undirected graphs, as symmetric digraphs without loops.
        If the edge (or an arc) already exists this operation changes its weight to a given one.
        """
        u, v = edge
        if u == v:
            raise ValueError("Loops are not allowed!")
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        self.weights[(u,v)] = weight
        if u not in self.graph[v]:
            self.graph[v].append(u)
        self.weights[(v,u)] = weight

    # reading from a file
    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Read the graph from file, that in each line contains either
        the description of a vertex (one word) or
        the description of an edge/arc (2 words) or
        the description of an weighted edge/arc (at least 3 words) or
        The resulting graph is returned as a neighbourhood list.
        Variable "filename" contains the whole path to the file.
        """
        graph = WeightedGraph()
        file = open(filename, "r")          # open the file to read
        for line in file:                   # for each line of the file
          words = line.strip().split()      # splits the line into words
          if len(words) == 1:               # one word - vertex description
            graph.add_vertex(words[0])
          elif len(words) == 2:             # two words, the edge description
            if directed:
              graph.add_arc([words[0], words[1]])
            else:
              graph.add_edge([words[0], words[1]])
          elif len(words) >= 3:             # at least three words, first three are the weighted edge description
            if directed:
              graph.add_arc([words[0], words[1]], int(words[2]))
            else:
              graph.add_edge([words[0], words[1]], int(words[2]))

        file.close()
        return graph

    def min_spanning_tree(self):
      """
      Jarnik-Prim algorithm
      """
      for v in self.graph:   # Choose any vertex of a graph
        break
      wtree = WeightedGraph({v:[]}, {})
      total = 0               # total weight
      q = PriorityQueue()
      for u in self.graph[v]:
        q.put((self.weights[(v,u)], v, u))
      while not q.empty():
        (w, v, u) = q.get()
        if u not in wtree.graph:
          total += w
          wtree.add_edge((v, u), w)
          for x in self.graph[u]:
            if not x in wtree.graph:
              q.put((self.weights[(u,x)], u, x))
      if len(wtree.graph) < len(self.graph):
        print("Graph is not connected. Returned is a tree for some connected component.")
      return total, wtree

    def to_matrix(self):
        """
        Converts the weighted graph to an adjacency matrix.
        Each element (i, j) in the matrix represents the weight of the edge from vertex i to vertex j.
        """
        vertices = self.vertices()
        n = len(vertices)
        matrix = np.inf * np.ones((n, n))  # Set all distances to infinity initially
        np.fill_diagonal(matrix, 0)  # Set diagonal to 0 (no distance from a vertex to itself)

        for u in self.graph:
            u_index = vertices.index(u)
            for v in self.graph[u]:
                v_index = vertices.index(v)
                matrix[u_index, v_index] = self.weights[(u,v)]  # Assign the weight to the matrix
        return matrix

    @staticmethod
    def from_matrix(matrix, vertices=None):
        """
        Converts an adjacency matrix back to a weighted graph.
        The matrix should have the same structure as returned by `to_matrix`.
        """
        n = len(matrix)
        if vertices is None:
            vertices = [*range(n)]
        wgraph = WeightedGraph()
        for i in range(n):
            for j in range(n):
                if matrix[i, j] != np.inf and i != j:
                    wgraph.add_arc([vertices[i], vertices[j]], matrix[i, j])
        return wgraph

    def floyd_warshall(self):
        """
        Computes the shortest paths between all pairs of vertices using the Floyd-Warshall algorithm.
        Returns a distance matrix where dist[i][j] is the shortest distance from vertex i to vertex j.
        """

        dist_matrix = self.to_matrix()
        n = len(dist_matrix)

        for k in range(n):
            for i in range(n):
                for j in range(n):
                    if dist_matrix[i, j] > dist_matrix[i, k] + dist_matrix[k, j]:
                        dist_matrix[i, j] = dist_matrix[i, k] + dist_matrix[k, j]
        return dist_matrix


    def Dijkstra(self, s):
      """
      Dijkstra ALgorithm - computes shortest paths from s to all other vertices.
      Return two dictionaries (of distances and of precessors). \
      """
      # Init
      dist = {}
      pred = {}
      for v in self.graph:
        dist[v] = 2**31   # substitute of intifine
        pred[v] = None
      dist[s] = 0
      q = PriorityQueue()
      q.put((0, s))
      #V = set()           # set of processed vertices
      while not q.empty():
        (d, u) = q.get()
        if dist[u] >= d:
          #V.add(u)
          for v in self.graph[u]:    # relax
            w = self.weights[(u,v)]
            if dist[v] > dist[u] + w:
              dist[v] = dist[u] + w
              pred[v] = u
              q.put((dist[v], v))
      return dist, pred


## Use of code

### Part 7

Strongly connected components (homework).

In [5]:
graph = Graph()
graph.add_arc(['a', 'b'])
graph.add_arc(['b', 'c'])
graph.add_arc(['c', 'd'])
graph.add_arc(['d', 'e'])
graph.add_arc(['e', 'f'])
graph.add_arc(['e', 'c'])

In [6]:
print(graph.strong_connected_components())

[{'a'}, {'b'}, {'c', 'd', 'e'}, {'f'}]


In [7]:
!wget https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt

--2024-12-05 00:09:08--  https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt [following]
--2024-12-05 00:09:09--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [text/plain]
Saving to: ‘weighted0.txt.3’

weighted0.txt.3     100%[===================>]      99  --.-KB/s    in 0s      

2024-12-05 00:09:09 (5.30 MB/s) - ‘weighted0.txt.3’ saved [99/99]



In [8]:
wgraph = WeightedGraph.from_edges("weighted0.txt")

In [9]:
print(wgraph)

A: (B, 3) (E, 10)
B: (A, 3) (C, 26) (D, 12)
E: (A, 10) (D, 7) (F, 8) (H, 4)
C: (B, 26) (D, 17) (F, 13) (G, 14)
D: (B, 12) (C, 17) (E, 7) (F, 15)
F: (C, 13) (D, 15) (E, 8) (G, 9) (H, 6)
G: (C, 14) (F, 9) (H, 16) (I, 11)
H: (E, 4) (F, 6) (G, 16)
I: (G, 11)



In [10]:
dist, pred = wgraph.Dijkstra('A')

In [11]:
print(dist)
print(pred)

{'A': 0, 'B': 3, 'E': 10, 'C': 29, 'D': 15, 'F': 18, 'G': 27, 'H': 14, 'I': 38}
{'A': None, 'B': 'A', 'E': 'A', 'C': 'B', 'D': 'B', 'F': 'E', 'G': 'F', 'H': 'E', 'I': 'G'}


In [12]:
!wget https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted1.txt

--2024-12-05 00:09:09--  https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted1.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted1.txt [following]
--2024-12-05 00:09:09--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 922 [text/plain]
Saving to: ‘weighted1.txt.1’

weighted1.txt.1     100%[===================>]     922  --.-KB/s    in 0s      

2024-12-05 00:09:09 (42.0 MB/s) - ‘weighted1.txt.1’ saved [922/922]



In [13]:
wgraph2 = WeightedGraph.from_edges("weighted1.txt")

In [14]:
print(len(wgraph2.vertices()))

25


In [67]:
%timeit wgraph2.Dijkstra('1')

206 µs ± 5.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [69]:
%timeit wgraph2.floyd_warshall()

11.8 ms ± 3.66 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Part 6 (Weigthed graphs)

In [17]:
wgraph = WeightedGraph()
wgraph.add_edge(("a", "b"), 2)

print(wgraph)

a: (b, 2)
b: (a, 2)



In [18]:
!wget https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt

--2024-12-05 00:09:22--  https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt [following]
--2024-12-05 00:09:22--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [text/plain]
Saving to: ‘weighted0.txt.4’

weighted0.txt.4     100%[===================>]      99  --.-KB/s    in 0s      

2024-12-05 00:09:22 (6.51 MB/s) - ‘weighted0.txt.4’ saved [99/99]



In [19]:
%cat weighted0.txt

A B 3
A E 10
B C 26
B D 12
C D 17
C F 13
C G 14
D E 7
D F 15
E F 8
E H 4
F G 9
F H 6
G H 16
G I 11


In [20]:
wgraph = WeightedGraph.from_edges("weighted0.txt")
print(wgraph)

A: (B, 3) (E, 10)
B: (A, 3) (C, 26) (D, 12)
E: (A, 10) (D, 7) (F, 8) (H, 4)
C: (B, 26) (D, 17) (F, 13) (G, 14)
D: (B, 12) (C, 17) (E, 7) (F, 15)
F: (C, 13) (D, 15) (E, 8) (G, 9) (H, 6)
G: (C, 14) (F, 9) (H, 16) (I, 11)
H: (E, 4) (F, 6) (G, 16)
I: (G, 11)



Example of vertex removal.

In [21]:
wgraph.del_vertex('H')
print(wgraph)

A: (B, 3) (E, 10)
B: (A, 3) (C, 26) (D, 12)
E: (A, 10) (D, 7) (F, 8)
C: (B, 26) (D, 17) (F, 13) (G, 14)
D: (B, 12) (C, 17) (E, 7) (F, 15)
F: (C, 13) (D, 15) (E, 8) (G, 9)
G: (C, 14) (F, 9) (I, 11)
I: (G, 11)



In [22]:
w, wtree = wgraph.min_spanning_tree()
print(w)
print(wtree)

61
A: (B, 3) (E, 10)
B: (A, 3)
E: (A, 10) (D, 7) (F, 8)
D: (E, 7)
F: (E, 8) (G, 9) (C, 13)
G: (F, 9) (I, 11)
I: (G, 11)
C: (F, 13)



### Part 5 (Distances via BFS)

In [23]:
%%writefile edges.txt
1 2
1 3
1 6
2 4
2 5
3 5
3 6
3 7
4 5
4 8
5 7
7 8

Writing edges.txt


In [24]:
graph1 = Graph.from_edges("edges.txt")
print(graph1)

1: 2 3 6
2: 1 4 5
3: 1 5 6 7
6: 1 3
4: 2 5 8
5: 2 3 4 7
7: 3 5 8
8: 4 7



In [25]:
dist = graph1.distance("1")
print(dist)

{'1': 0, '2': 1, '3': 1, '6': 1, '4': 2, '5': 2, '7': 2, '8': 3}


#### Milgram's experiment &mdash; Small World Phenomenon

We will use graph in $G(n,p)$ (namely $G(2000, 1/300)$) as an illustration.

In [26]:
# to obey an issue with recursion depth (provided it happens - for Colab now it does not)
import sys

n = 2000
p = 1/300
sys.setrecursionlimit(n+5)

rgraph = Graph.random_graph(n, p)
graph = rgraph.connected_components_graphs()[0]
print(len(graph.vertices()))

1995


In [27]:
%timeit graph = rgraph.connected_components_graphs()[0]

3.36 ms ± 786 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit graph = rgraph.ConnectedComponentsBFS()[0]
print(len(graph.vertices()))

4.51 ms ± 574 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1995


In [29]:
%timeit graph = rgraph.ConnectedComponentsGraphs()[0]
print(len(graph.vertices()))

3.05 ms ± 96.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1995


In [30]:
md = {}
ecc = {}
for v in graph.vertices():
  dist = graph.distance(v)
  ecc[v] = max(dist.values())
  md[v] = sum(dist.values())/len(dist.values())
print("Radius: ", min(ecc.values()), "Diameter: ", max(ecc.values()), "Average: ", sum(md.values())/len(md.values()))

Radius:  5 Diameter:  8 Average:  4.210614254935578


### Part 4 (Connected components via DFS)

In [31]:
graph1 = Graph.random_graph(10, 1/5)
print(graph1)

1: 3
2: 3 4 8 9
3: 1 2 8 9
4: 2 6
5:
6: 4
7: 8 9
8: 2 3 7 10
9: 2 3 7
10: 8



In [32]:
print(len(graph1.connected_components()))
print(graph1.connected_components())

3
[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}, {1, 2, 3, 4, 6, 7, 8, 9, 10}, {5}]


In [33]:
graphlist = graph1.connected_components_graphs()
print(len(graphlist))

2


In [34]:
print(graphlist[0])

1: 3
2: 3 4 8 9
3: 1 2 8 9
4: 2 6
6: 4
7: 8 9
8: 2 3 7 10
9: 2 3 7
10: 8



In [35]:
print(len(graph1.ConnectedComponentsBFS()))
print(graph1.ConnectedComponentsBFS()[0])

2
1: 3
2: 3 4 8 9
3: 1 2 8 9
4: 2 6
6: 4
7: 8 9
8: 2 3 7 10
9: 2 3 7
10: 8



In [36]:
print(graph1.ConnectedComponentsGraphs()[0])

1: 3
3: 1 2 8 9
9: 2 3 7
7: 8 9
8: 2 3 7 10
10: 8
2: 3 4 8 9
4: 2 6
6: 4



### Part 3 (Prufer codes)

In [37]:
%%writefile tree1.txt
1 3
3 5
2 5
4 5
6 5

Writing tree1.txt


In [38]:
tree1 = Graph.from_edges("tree1.txt")

In [39]:
print(tree1)

1: 3
3: 1 5
5: 3 2 4 6
2: 5
4: 5
6: 5



In [40]:
print(tree1.Prufer())

3 5 5 5


In [41]:
print(Graph.tree_from_Prufer("3 5 5 5"))

1: 3
2: 5
3: 1 5
4: 5
5: 2 3 4 6
6: 5



In [42]:
print(Graph.tree_from_Prufer("5 4 3 1"))

1: 3 6
2: 5
3: 4 1
4: 5 3
5: 2 4
6: 1



In [43]:
print(Graph.tree_from_Prufer("1 1 1 2 3"))

1: 4 5 6 2
2: 1 3
3: 2 7
4: 1
5: 1
6: 1
7: 3



### Part2

In [44]:
%%writefile edges.txt
a b
b c
b d
d
d c
e
f


Overwriting edges.txt


In [45]:
%cat edges.txt

a b
b c
b d
d
d c
e
f


In [46]:
graph2 = Graph.from_edges("edges.txt")
print(graph2)

a: b
b: a c d
c: b d
d: b c
e:
f:



In [47]:
graph2.to_neighbourlist("neighbourhood.txt")

In [48]:
%cat "neighbourhood.txt"

a: b
b: a c d
c: b d
d: b c
e:
f:


In [49]:
!wget https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt

--2024-12-05 00:09:36--  https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt [following]
--2024-12-05 00:09:36--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [text/plain]
Saving to: ‘weighted0.txt.5’

weighted0.txt.5     100%[===================>]      99  --.-KB/s    in 0s      

2024-12-05 00:09:36 (5.01 MB/s) - ‘weighted0.txt.5’ saved [99/99]



In [50]:
graph3 = Graph.from_edges("weighted0.txt")
print(graph3)

A: B E
B: A C D
E: A D F H
C: B D F G
D: B C E F
F: C D E G H
G: C F H I
H: E F G
I: G



In [51]:
# it works as well
!wget https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/refs/heads/main/weighted0.txt

--2024-12-05 00:09:36--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/refs/heads/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [text/plain]
Saving to: ‘weighted0.txt.6’

weighted0.txt.6     100%[===================>]      99  --.-KB/s    in 0s      

2024-12-05 00:09:36 (3.25 MB/s) - ‘weighted0.txt.6’ saved [99/99]



In [52]:
graph4 = Graph.random_graph(10, 1/3)
print(graph4)

1: 2 3 4 6 7 8 10
2: 1 3 6 8
3: 1 2 6 7 8 10
4: 1
5:
6: 1 2 3 8 9
7: 1 3 10
8: 1 2 3 6 9
9: 6 8
10: 1 3 7



In [53]:
print(Graph.cycle(6))

1: 2 6
2: 1 3
3: 2 4
4: 3 5
5: 4 6
6: 5 1



### Part1

In [54]:
vertices = ["a", "b", "c", "d"]
matrix = np.array([[0,1,0,0],[1,0,1,0],[0,1,0,1],[0,0,0,1]])
print(vertices)
print(matrix)
print("---------------------------")
print_matrix(vertices, matrix)
print("---------------------------")
print_matrix(None,matrix)

['a', 'b', 'c', 'd']
[[0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]]
---------------------------
a :  b
b :  a  c
c :  b  d
d :  d
---------------------------
1 :  2
2 :  1  3
3 :  2  4
4 :  4


In [55]:
graph_dict = {
  "a": ["b"],
  "b": ["a", "c"],
  "c": ["b", "d"],
  "d": ["c"]
}
print(graph_dict)
print("---------------------------")
print_dict(graph_dict)

{'a': ['b'], 'b': ['a', 'c'], 'c': ['b', 'd'], 'd': ['c']}
---------------------------
a :  b
b :  a  c
c :  b  d
d :  c


In [56]:
graph1 = Graph.from_matrix(matrix, vertices)
print(graph1)

a: b
b: a c
c: b d
d: d



In [57]:
print(Graph(graph_dict))  # the same result

a: b
b: a c
c: b d
d: c



In [58]:
print(graph1.vertices())

['a', 'b', 'c', 'd']


In [59]:
print(graph1.matrix())

[[0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]]


In [60]:
graph1.add_vertex("e")
print(graph1)

a: b
b: a c
c: b d
d: d
e:



In [61]:
graph1.add_edge(["e", "f"])
print(graph1)

a: b
b: a c
c: b d
d: d
e: f
f: e



In [62]:
graph1.add_arc(["e", "a"])  # breaking the symmetry
print(graph1)

a: b
b: a c
c: b d
d: d
e: f a
f: e



In [63]:
graph1.add_edge(["e", "a"])  # restoring the symmetry
print(graph1)

a: b e
b: a c
c: b d
d: d
e: f a
f: e



In [64]:
graph1.add_edge(["e", "f"]) # do nothing, an edge already exists
print(graph1)

a: b e
b: a c
c: b d
d: d
e: f a
f: e



In [65]:
graph1.del_vertex("f")  # removing a vertex
print(graph1)

a: b e
b: a c
c: b d
d: d
e: a



In [66]:
graph1.add_edge(["e", "e"]) # an error

ValueError: Loops are not allowed!

In [ ]:
graph1.add_arc(["e", "e"]) # OK - loops are allowed in digraphs
print(graph1)

In [ ]:
print(Graph.cycle(6))

In [ ]:
#  1. preorder(v) and postorder(v) Functions

# In[2]:


# In preorder, we visit the current node first, then recursively visit each child node.

def preorder(v, tree, visited=None):
    if visited is None:
        visited = set()
    visited.add(v)
    print(v, end=' ')
    for neighbor in tree[v]:
        if neighbor not in visited:
            preorder(neighbor, tree, visited)

#In postorder, we visit all child nodes before visiting the current node.
def postorder(v, tree, visited=None):
    if visited is None:
        visited = set()
    visited.add(v)
    for neighbor in tree[v]:
        if neighbor not in visited:
            postorder(neighbor, tree, visited)
    print(v, end=' ')

tree = {
    1: [2, 3],
    2: [4, 5],
    3: [],
    4: [],
    5: []
}
print("Preorder traversal:")
preorder(1, tree)
print("\n")  # Adds a newline for readability

print("Postorder traversal:")
postorder(1, tree)
print("\n")
